# 第28章 类代码编写细节

## class语句

- class语句是对象的创建者并且是一个隐含的赋值运算——执行时，它会产生类对象，并把其引用值存储在前面所使用的变量名
- class语句也是真正的可执行代码，直到Python抵达并运行定义的class语句前，你的类都不存在

- 当class语句自身运行时（不是稍后调用类来创建实例的时候），class语句内的所有语句都会执行，在class语句内赋值的变量名，会创建类属性，而内嵌的def则会创建类方法

In [1]:
class MixedNames:
    data = 'spam'
    def __init__(self, value):
        self.data = value
    def display(self):
        print self.data, MixedNames.data

x = MixedNames(1)
y = MixedNames(2)
x.display()
y.display()

1 spam
2 spam


## 方法

- 方法位于class语句的主体内，是由def语句建立的函数对象；从抽象的视角来看，方法替实例对象提供了要继承的行为，从程序设计的角度来看，方法的工作方式和简单函数完全一致，只是有个重要差异：方法的第一个参数总是接收方法调用的隐性主体，也就是实例对象
- 在类方法中，按惯例第一个参数通常都称为self（严格地说，只有其位置重要，而不是它的名称），这个参数给方法提供了一个钩子，从而返回调用的主体，也就是实例对象：因为类可以产生许多实例对象，所以需要这个参数来管理每个实例彼此各不相同的数据

- 方法一般是通过实例调用的，不过，通过类调用方法也扮演了一些特殊的角色

```python
# 调用超类构造函数
# 这种通过类调用方法的模式，是扩展继承方法行为（而不是完全取代）的一般基础
class Super:
    def __init__(self, x):
        ...default code...

class Sub(Super):
    def __init__(self, x, y):
        Super.__init__(self, x)
        ...custom code...
        
I = Sub(1, 2)
```

## 继承

- 在Python中，当对对象进行点号运算时，就会发生继承，而且涉及了搜索属性定义树（一或多个命名空间）；每次使用`object.attr`形式的表达式时（object是实例或类对象），Python会从头到尾搜索命名空间树，先从对象开始，寻找所能找到的第一个attr，这包括在方法中对self属性的引用；因为树中较低的定义会覆盖较高的定义，继承构成了专有化的基础

### 属性树的构造

- 实力属性是由对方法内self属性进行赋值运算而生成的
- 类属性是通过class语句内的语句（赋值语句）而生成的
- 超类的连接是通过class语句首行的括号内列出类而生成的

### 继承方法的专有化

- 子类可以完全取代继承的属性，提供超类可以找到的属性，也可以通过已覆盖的方法回调超类来扩展超类的方法

In [2]:
class Super:
    def method(self):
        print "in Super.method"
    def delegate(self):
        self.action()
        
class Inheritor(Super):
    pass

class Replacer(Super):
    def method(self):
        print "in Replacer.method"

class Extender(Super):
    def method(self):
        print "starting Extender.method"
        Super.method(self)
        print "ending Extender.method"
        
class Provider(Super):
    def action(self):
        print "in Provider.action"
        
for klass in (Inheritor, Replacer, Extender):
    print "\n" + klass.__name__ + '...'
    klass().method()
    
print "\nProvider..."
x = Provider()
x.delegate()


Inheritor...
in Super.method

Replacer...
in Replacer.method

Extender...
starting Extender.method
in Super.method
ending Extender.method

Provider...
in Provider.action


### 抽象超类

- 注意上一个例子中的provider类是如何工作的，这种“填空”的代码结构一般就是OOP的软件框架，至少，从delegate方法的角度来看，这个例子中的超类有时也称作是抽象超类——也就是类的部分行为默认是由其子类所提供的，如果预期的方法没有在子类中定义，当继承搜索失败时，Python会引发未定义变量名的异常

## 命名空间词典

- 之前我们说过模块的命名空间实际上是以字典的形式实现的，并且可以由内置属性`__dict__`显示这一点，类和实例对象也是如此：属性点号运算其实内部就是字典的索引运算，而属性继承其实就是搜索链接的字典而已，实际上，实例和类对象就是Python中带有链接的字典而已
- 当我们制作子类的实例时，该实例一开始会是空的命名空间字典，但是有链接会指向它的类，让继承搜索能顺着寻找
- 实例中有个`__class__`属性链接到了它的类，而类有个`__bases__`属性，是一个元组，其中包含了通往更高的超类的链接